# Динамика доли каналов в течение дня
Пример расчета доли в разбивке по часам

## Описание задачи и условий расчета
- Период: 01.09.2022 - 31.10.2022 по месяцам
- Временной интервал: 05:00-29:00 с разбивкой по часам
- ЦА: 4+ и 25-49
- Место просмотра: Все места (дом+дача)
- Каналы: Рен ТВ, ТНТ									
- Статистики: Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [2]:
# выберем телесети по имени (части имени)
cats.get_tv_net(name=["РЕН ТВ", "ТНТ"])

# далее в запросах будем использовать id телесетей.

Запрошены записи: 0 - 0
Всего найдено записей: 0



""


In [3]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['возраст'])

Запрошены записи: 0 - 125
Всего найдено записей: 125



,id,valueId,name,colName,valueName
0,2,33,Возраст,Age,33
1,16,1,Возраст детей: До 1 года,KidsAge2,Да
2,16,0,Возраст детей: До 1 года,KidsAge2,Нет
3,20,0,Возраст детей: 7-11 лет,KidsAge6,Нет
4,20,1,Возраст детей: 7-11 лет,KidsAge6,Да
...,...,...,...,...,...
120,2,10,Возраст,Age,10
121,2,89,Возраст,Age,89
122,2,8,Возраст,Age,8
123,18,0,Возраст детей: 2-3 года,KidsAge4,Нет


## Формирование задания
Зададим условия расчета

In [4]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-09-01', '2022-10-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId IN (60, 83)'

# Указываем список статистик для расчета
statistics = ['Share']

# Указываем срезы
slices = ['tvNetName','researchMonth', 'timeBand60']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Формируем группу значений параметров для различных ЦА

In [5]:
# Задаем ЦА по которым будет рассчитан отчет
targets = {
    '4+':'age >= 4',
    '25-49':'age >= 25 AND age <= 49'
}

## Расчет задания

In [6]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задания и отправляем на расчет
for target, logical in targets.items():
    
    # Формируем задание на расчет ЦА
    project_name = f"{target}" 
    basedemo_filter = f"{logical}"
      
    # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

Отправляем задания на расчет
..
Ждем выполнения
Расчет задач (2) [ = = = = = = = = = = = ] время расчета: 0:00:36.699300
Расчет завершен, получаем результат
Собираем таблицу
..

Настраиваем внешний вид таблицы

In [7]:
df = df.astype(dtype={'timeBand60':int}) #меняем формат для корректной сортировки

df_res = pd.pivot_table(df, values=['Share'],
                        index=['timeBand60'], 
                        columns=['tvNetName','prj_name','researchMonth'])
df_res

Share                                              \
tvNetName         РЕН ТВ                                         ТНТ   
prj_name           25-49                    4+                 25-49   
researchMonth 2022-09-01 2022-10-01 2022-09-01 2022-10-01 2022-09-01   
timeBand60                                                             
50000           7.748215   8.213033   6.923507   6.756382   4.554789   
60000           5.765206   5.820982   5.935847   5.900041   5.207848   
70000           8.425078   8.172862   7.179436   7.310552   3.911889   
80000          10.023168   9.938383   7.650761   8.071939   3.749406   
90000           9.159407   9.019961   7.204925   7.490215   4.430251   
100000          8.132017   8.423197   6.870419   7.240994   5.350718   
110000          6.725959   6.902629   5.976175   6.428518   5.670796   
120000          8.071337   8.272403   6.828929   7.169459   6.286750   
130000          6.969144   7.367133   5.940007   6.268319   6.665173   
140000          6.704917   7.215109   5.960554   6.325938   6.776809   
150000          6.623561   7.072815   5.800648   6.145854   6.429133   
160000          7.109449   7.730858   6.178328   6.571240   5.856000   
170000          6.642351   6.961744   6.239612   6.456166   5.533595   
180000          6.211249   6.398121   5.416907   5.405332   5.494947   
190000          7.437466   7.876928   5.520797   5.512759   6.695122   
200000          7.497998   7.473481   4.723323   4.623386   7.922633   
210000          7.902802   7.698890   4.817578   4.693685   7.310343   
220000          7.958745   7.683742   4.800329   4.632242   6.620538   
230000          8.684886   8.801728   5.319070   5.323419   6.487683   
240000          7.790497   8.459525   5.051915   5.190100   5.859760   
250000          8.283086   8.644031   5.395466   5.313272   5.802222   
260000          8.860136   9.874034   6.008031   6.119782   6.332713   
270000          8.682179   9.716180   5.897354   6.345936   5.969912   
280000          8.072651   9.100937   5.862570   6.565687   6.185095   

                                                
tvNetName                                       
prj_name                         4+             
researchMonth 2022-10-01 2022-09-01 2022-10-01  
timeBand60                                      
50000           5.023107   2.475619   2.560992  
60000           5.082872   2.796222   2.800176  
70000           4.276782   2.111617   2.222017  
80000           4.466676   2.030722   2.199666  
90000           4.294909   2.125448   2.059960  
100000          4.851785   2.480145   2.344439  
110000          5.372675   2.644455   2.527254  
120000          5.585278   2.948335   2.657460  
130000          5.928588   3.061868   2.795319  
140000          6.158472   3.126950   2.836800  
150000          6.417573   3.064705   2.975727  
160000          6.035512   2.883500   2.794153  
170000          6.063364   2.698446   2.826834  
180000          6.065314   2.660391   2.805131  
190000          7.363857   3.212633   3.507075  
200000          7.962562   3.915465   4.043411  
210000          7.657207   3.580799   3.780204  
220000          6.331706   3.257062   3.081146  
230000          5.712415   3.188135   2.846964  
240000          5.501911   2.827246   2.729116  
250000          5.400193   3.112974   2.900259  
260000          5.836924   3.304118   3.180170  
270000          5.914724   2.976716   2.972026  
280000          5.932185   3.062486   2.825433

## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [8]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('05_share_during_the_day'))
df_info = mtask.task_builder.get_report_info()
df_res.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()